#Домашняя работа № 3. Гибридные рекомендательные системы

##Набор данных MovieLens объемом 100 тыс.

Посмотрим на оригинальный датасет

In [194]:
import pandas as pd
import numpy as np

In [195]:
# загрузим данные
!wget 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

--2025-02-02 13:39:05--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  9.14MB/s    in 0.5s    

2025-02-02 13:39:06 (9.14 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]



In [196]:
# распакуем
import zipfile

with zipfile.ZipFile('/content/ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall('unpacked_files')

In [197]:
# прочитаем описание README
"""
ПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ
==============================================

Вот краткое описание данных.

ml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:
                gunzip ml-data.tar.gz
                tar xvf ml-data.tar
                mku.sh

u.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.
         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы
         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.
         Это разделенный табуляцией
         список идентификаторов пользователей | элементов | оценок | временных меток.
         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix

u.info - Количество пользователей, элементов и оценок в наборе данных u.

u.item - Информация о фильмах, которые будут показаны в этом разделе;
         список фильмов, которые будут показаны в виде вкладок:
         идентификатор фильма | название фильма | дата выхода | дата выхода видео |
         URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
         Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
         Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
         Триллер | Военный | Вестерн |
         Последние 19 полей обозначают жанры, 1 указывает на фильм относится к этому жанру,
         0 означает, что это не так; фильмы могут быть в сразу в нескольких жанрах.
         Идентификаторы фильмов используются в наборе данных u.data.

u.genre - список жанров.

u.user - Демографическая информация о пользователях; это вкладка разделенный список
         идентификатор пользователя | возраст | пол | род занятий | почтовый индекс.
         Идентификаторы пользователей используются в наборе данных u.data.

u.occupation - Список профессий.

u1.base - Наборы данных u1.base и u1.test через u5.base и u5.test
u1.test представляют собой 80% /20% разбиений данных u на обучающие и тестовые данные.
u2.base Каждый из наборов тестов u1, ..., u5 имеет непересекающиеся наборы тестов;
        это если для
u2.test 5-кратной перекрестной проверки (где вы повторяете свой эксперимент
        u3.base для каждого обучающего и тестового набора и усредняете результаты).
u3.test Эти наборы данных могут быть сгенерированы из u.data с помощью mku.sh.
u4.base
u4.test
u5.base
u5.test

ua.base - Наборы данных ua.base, ua.test, ub.base и ub.test
ua.test разделяют данные u на обучающий набор и тестовый набор с
ub.base ровно по 10 оценок для каждого пользователя в тестовом наборе.  Наборы
ub.test ua.test и ub.test не пересекаются.  Эти наборы данных могут
        быть сгенерированы из u.data с помощью mku.sh.

allbut.pl -- Скрипта, который генерирует обучающие и тестовые наборы, где
             все оценки пользователей, кроме n, содержатся в обучающих данных.

mku.sh -- Командный скрипт для генерации всех u наборов данных из u.data.

"""

'\nПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ\n==============================================\n\nВот краткое описание данных.\n\nml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:\n                gunzip ml-data.tar.gz\n                tar xvf ml-data.tar\n                mku.sh\n\nu.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.\n         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы\n         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.\n         Это разделенный табуляцией \n         список идентификаторов пользователей | элементов | оценок | временных меток. \n         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix   \n\nu.info - Количество пользователей, элементов и оценок в наборе данных u.\n\nu.item - Информация о фильмах, которые будут показаны в этом разделе;\n         список фильмов, которые будут показаны в виде вкладок: \n

In [198]:
# прочитаем
u_data = pd.read_csv('/content/unpacked_files/ml-100k/u.data', delimiter='\t')
u_data.columns = ['userID', 'movieID', 'rating', 'time']
u_info = pd.read_csv('/content/unpacked_files/ml-100k/u.info')

# так читать не хочет
# u_item = pd.read_csv('/content/unpacked_files/ml-100k/u.item')
# попробуем по-другому
import csv
csv_file = open('/content/unpacked_files/ml-100k/u.item', newline='', encoding='latin-1')
u_item = pd.read_csv(csv_file, delimiter='|')
u_item.columns = ['movieID', 'movie_title', 'release_date', 'video_release',
                  'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                  'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                  'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                  'Thriller', 'War', 'Western']
u_genre = pd.read_csv('/content/unpacked_files/ml-100k/u.genre', delimiter='|')
u_genre.columns = ['genre', '#']
u_user = pd.read_csv('/content/unpacked_files/ml-100k/u.user', delimiter='|')
u_user.columns = ['userID', 'age', 'gender', 'occupation', 'p_code']
u_occupation = pd.read_csv('/content/unpacked_files/ml-100k/u.occupation')
u_occupation.columns = ['occupation']

In [199]:
# пользователь | элемент | рейтинг | временная метка
u_data.head()

,userID,movieID,rating,time
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [200]:
u_info.head()

,943 users
0,1682 items
1,100000 ratings


In [201]:
# идентификатор фильма | название фильма | дата выхода | дата выхода видео |
# URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
# Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
# Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
# Триллер | Военный | Вестерн |
u_item.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [202]:
u_item_2 = u_item.copy()
u_item_2.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
def genre_to_string (str, num):
  if num == 1:
    string = str

  return

In [243]:
 # добавим единую колонку жанров
 cols = ['unknown', 'Action', 'Adventure', 'Animation', 'Children_s', 'Comedy',
         'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
         'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

for index, row in u_item_2.iterrows():
    string = str()
    for ele in cols:
        if row[ele] == 1:
            if string == '':
                string += ele
            else:
                string += ('|'+ele)
        else:
            continue
    u_item.loc[index, 'genre'] = string

In [244]:
u_item.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,Action|Adventure|Thriller
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action|Comedy|Drama
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Crime|Drama|Thriller
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama


In [206]:
# посмотрим на список жанров
u_genre

,genre,#
0,Action,1
1,Adventure,2
2,Animation,3
3,Children's,4
4,Comedy,5
5,Crime,6
6,Documentary,7
7,Drama,8
8,Fantasy,9
9,Film-Noir,10


In [207]:
# пользователь | возраст | пол | род занятий | почтовый индекс
u_user.head()

,userID,age,gender,occupation,p_code
0,2,53,F,other,94043
1,3,23,M,writer,32067
2,4,24,M,technician,43537
3,5,33,F,other,15213
4,6,42,M,executive,98101


In [208]:
# профессии
u_occupation.head()

,occupation
0,artist
1,doctor
2,educator
3,engineer
4,entertainment


##LightFM (default)

In [17]:
import numpy as np

In [122]:
!pip install lightfm

In [19]:
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [335]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [338]:
data = fetch_movielens(min_rating=5.0)
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 19048 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 2153 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [21]:
train = movielens['train']
test = movielens['test']

Посмотрим на все это

In [22]:
train.toarray(), len(train.toarray())

(array([[5, 3, 4, ..., 0, 0, 0],
        [4, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [5, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 5, 0, ..., 0, 0, 0]], dtype=int32),
 943)

In [23]:
test.toarray(), len(test.toarray())

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 943)

In [24]:
data = fetch_movielens()
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 90570 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 9430 stored elements in COOrdinate format>


In [25]:
# зададим рейтинг
data = fetch_movielens(min_rating=5.0)
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 2153 stored elements in COOrdinate format>


In [26]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM()
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.44, test 0.13.


In [27]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.60, test 0.22.


In [28]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [29]:
sample_recommendation(model, movielens, [10, 25, 451])

User 10
     Known positives:
        B
        D
        S
     Recommended:
        R
        S
        E
User 25
     Known positives:
        T
        T
        D
     Recommended:
        S
        L
        C
User 451
     Known positives:
        T
        B
        P
     Recommended:
        R
        A
        S


##Соберем свой датафрейм

In [245]:
# объединим u_data и u_user
df_1 = u_data.merge(u_user, on='userID').reset_index(drop=True)
df_1.head()

,userID,movieID,rating,time,age,gender,occupation,p_code
0,186,302,3,891717742,39,F,executive,00000
1,22,377,1,878887116,25,M,writer,40206
2,244,51,2,880606923,28,M,technician,80525
3,166,346,1,886397596,47,M,educator,55113
4,298,474,4,884182806,44,M,executive,01581


In [246]:
# объединим df_1 и u_item
df_2 = df_1.merge(u_item, on='movieID').reset_index(drop=True)
df_2.head()

,userID,movieID,rating,time,age,gender,occupation,p_code,movie_title,release_date,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,3,891717742,39,F,executive,00000,L.A. Confidential (1997),01-Jan-1997,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,1,878887116,25,M,writer,40206,Heavyweights (1994),01-Jan-1994,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,2,880606923,28,M,technician,80525,Legends of the Fall (1994),01-Jan-1994,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,1,886397596,47,M,educator,55113,Jackie Brown (1997),01-Jan-1997,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,4,884182806,44,M,executive,01581,Dr. Strangelove or: How I Learned to Stop Worr...,01-Jan-1963,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [247]:
# выберем самое интересное
cols = ['userID', 'movieID', 'movie_title', 'rating', 'age', 'gender', 'occupation',
        'unknown', 'Action', 'Adventure', 'Animation',
        'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
        'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
        'Thriller', 'War', 'Western', 'genre']
df_data = df_2[cols]
df_data.head()

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [212]:
# проверим жанр unknown
df_data['unknown'].sum()

9

In [213]:
# посмотрим на эти строки
df_data.loc[df_data['unknown'] == 1]

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
2144,130,267,unknown,5,20,M,none,1,0,0,...,0,0,0,0,0,0,0,0,0,
3732,5,267,unknown,4,33,F,other,1,0,0,...,0,0,0,0,0,0,0,0,0,
7151,268,267,unknown,3,24,M,engineer,1,0,0,...,0,0,0,0,0,0,0,0,0,
8465,181,1373,Good Morning (1971),1,26,M,executive,1,0,0,...,0,0,0,0,0,0,0,0,0,
12332,297,267,unknown,3,29,F,educator,1,0,0,...,0,0,0,0,0,0,0,0,0,
14595,319,267,unknown,4,38,M,programmer,1,0,0,...,0,0,0,0,0,0,0,0,0,
48878,532,267,unknown,3,20,M,student,1,0,0,...,0,0,0,0,0,0,0,0,0,
92840,833,267,unknown,1,34,M,writer,1,0,0,...,0,0,0,0,0,0,0,0,0,
99000,422,267,unknown,4,26,M,entertainment,1,0,0,...,0,0,0,0,0,0,0,0,0,


In [214]:
# считаю целесообразным удалить эти строки
df_data.drop(df_data.index[df_data['unknown'] == 1], axis=0, inplace=True)

<ipython-input-214-eb2aae18e79a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.drop(df_data.index[df_data['unknown'] == 1], axis=0, inplace=True)


In [215]:
# проверим
df_data.loc[df_data['unknown'] == 1]

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre


In [248]:
# соберем список уникальных профессий
list_occupations_unique = list(df_data['occupation'].drop_duplicates())
# посмотрим
list_occupations_unique,  len(list_occupations_unique)

(['executive',
  'writer',
  'technician',
  'educator',
  'engineer',
  'librarian',
  'programmer',
  'administrator',
  'student',
  'retired',
  'other',
  'doctor',
  'marketing',
  'artist',
  'lawyer',
  'salesman',
  'homemaker',
  'healthcare',
  'none',
  'entertainment',
  'scientist'],
 21)

In [249]:
df_data.head()

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [251]:
#соберем объект жанров по фильмам
series_genre_movies = df_data['genre'].str.split('|')
# посмотрим
series_genre_movies[0], series_genre_movies, len(series_genre_movies)

(['Crime', 'Film-Noir', 'Mystery', 'Thriller'],
 0        [Crime, Film-Noir, Mystery, Thriller]
 1                         [Children_s, Comedy]
 2               [Drama, Romance, War, Western]
 3                               [Crime, Drama]
 4                                [Sci-Fi, War]
                          ...                  
 99271                                 [Comedy]
 99272                         [Comedy, Sci-Fi]
 99273                               [Thriller]
 99274                     [Children_s, Comedy]
 99275                                [Western]
 Name: genre, Length: 99276, dtype: object,
 99276)

##Более интересная модель

In [287]:
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from lightfm.datasets import fetch_movielens

In [252]:
# посмотрим на эти строки
df_data.loc[df_data['userID'] == 2144]

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre


In [241]:
df_data.reset_index()

,index,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,1,22,377,Heavyweights (1994),1,25,M,writer,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99262,99271,880,476,"First Wives Club, The (1996)",3,13,M,student,0,0,...,0,0,0,0,0,0,0,0,0,Comedy
99263,99272,716,204,Back to the Future (1985),5,36,F,administrator,0,0,...,0,0,0,0,0,1,0,0,0,Comedy|Sci-Fi
99264,99273,276,1090,Sliver (1993),1,21,M,student,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
99265,99274,13,225,101 Dalmatians (1996),2,47,M,educator,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy


In [253]:
df_data.head(2147)

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142,239,10,Richard III (1995),5,39,M,artist,0,0,0,...,0,0,0,0,0,0,0,1,0,Drama|War
2143,238,111,"Truth About Cats & Dogs, The (1996)",4,42,F,administrator,0,0,0,...,0,0,0,0,1,0,0,0,0,Comedy|Romance
2144,130,267,unknown,5,20,M,none,1,0,0,...,0,0,0,0,0,0,0,0,0,unknown
2145,90,662,Somewhere in Time (1980),5,60,M,educator,0,0,0,...,0,0,0,0,1,0,0,0,0,Drama|Romance


In [181]:
series_genre.isna().sum()

0

In [256]:
# соберем список уникальных жанров
series_genre = df_data['genre'].str.split('|')
list_genre_unique = list(set(np.concatenate(series_genre).ravel()))

list_genre_unique[:10], len(list_genre_unique)

(['Sci-Fi',
  'Western',
  'War',
  'Drama',
  'Children_s',
  'Animation',
  'Action',
  'Crime',
  'Film-Noir',
  'Adventure'],
 19)

In [257]:
# соберем список уникальных профессий
list_occupations_unique = list(df_data['occupation'].drop_duplicates())

list_occupations_unique[:5], len(list_occupations_unique)

(['executive', 'writer', 'technician', 'educator', 'engineer'], 21)

In [258]:
# соберем датасет
dataset = Dataset()
dataset.fit(users=df_data['userID'],
            items=df_data['movieID'],
            item_features=list_genre_unique,
            user_features=list_occupations_unique)

In [259]:
# соберем список уникальных пользователей и их профессии
df_unique_userID = df_data.drop_duplicates('userID')
list_user_features = [(x,[y]) for x,y in zip(df_unique_userID['userID'],
                      df_unique_userID['occupation'])
]

list_user_features[:10], len(list_user_features)

([(186, ['executive']),
  (22, ['writer']),
  (244, ['technician']),
  (166, ['educator']),
  (298, ['executive']),
  (115, ['engineer']),
  (253, ['librarian']),
  (305, ['programmer']),
  (6, ['executive']),
  (62, ['administrator'])],
 942)

In [260]:
# соберем список уникальных фильмов и их жанры
df_unique_movieID = df_data.drop_duplicates('movieID')
list_item_features = [(x,y) for x,y in zip(df_unique_movieID['movieID'], series_genre_movies)]

list_item_features[:10], len(list_item_features)

([(302, ['Crime', 'Film-Noir', 'Mystery', 'Thriller']),
  (377, ['Children_s', 'Comedy']),
  (51, ['Drama', 'Romance', 'War', 'Western']),
  (346, ['Crime', 'Drama']),
  (474, ['Sci-Fi', 'War']),
  (265, ['Action', 'Thriller']),
  (465, ['Adventure', 'Children_s', 'Romance']),
  (451, ['Comedy', 'Musical', 'Romance']),
  (86, ['Drama']),
  (257, ['Action', 'Adventure', 'Comedy', 'Sci-Fi'])],
 1681)

In [261]:
# подготовим sparse matrix
sm_user_features = dataset.build_user_features(list_user_features)
sm_user_features

<942x963 sparse matrix of type '<class 'numpy.float32'>'
	with 1884 stored elements in Compressed Sparse Row format>

In [262]:
# подготовим sparse matrix
sm_item_features = dataset.build_item_features(list_item_features)
sm_item_features

<1681x1700 sparse matrix of type '<class 'numpy.float32'>'
	with 5227 stored elements in Compressed Sparse Row format>

In [264]:
# подготовим данные о взаимодействии
sm_interactions, sm_weights = dataset.build_interactions(df_data[['userID',
                                                                  'movieID',
                                                                  'rating']].values)
sm_interactions

<942x1681 sparse matrix of type '<class 'numpy.int32'>'
	with 99276 stored elements in COOrdinate format>

Вместо использования исходных идентификаторов пользователей и элементов в нашем наборе данных LightFM внутренне присваивает каждому пользователю и элементу новый последовательный неотрицательный целочисленный идентификатор. Мы можем увидеть это сопоставление следующим образом:

In [273]:
user_id_map, user_feature_map, item_id_map, feature_item_map = dataset.mapping()

Посмотрим

In [279]:
list(user_id_map.items())[:3]

[(186, 0), (22, 1), (244, 2)]

In [280]:
list(user_feature_map.items())[:3]

[(186, 0), (22, 1), (244, 2)]

In [281]:
list(item_id_map.items())[:3]

[(302, 0), (377, 1), (51, 2)]

In [283]:
list(feature_item_map.items())[:3]

[(302, 0), (377, 1), (51, 2)]

In [317]:
# оценка эффективности
sm_train_interactions, sm_test_interactions = random_train_test_split(sm_interactions,
                                                                      test_percentage=0.2,
                                                                      random_state=42)
print(f'Shape of train interactions: {sm_train_interactions.shape}')
print(f'Shape of test interactions: {sm_test_interactions.shape}')

Shape of train interactions: (942, 1681)
Shape of test interactions: (942, 1681)


In [333]:
train.shape

(943, 1682)

In [334]:
test.shape

(943, 1682)

In [331]:
# построим модель
LEARNING_RATE = 0.05
EPOCHS = 10
COMPONENTS = 10
ITEM_ALPHA = 1e-6   # коэффициент упорядочения характеристик признаков
USER_ALPHA = 1e-6   # коэффициент упорядочения пользовательских функций

model = LightFM(loss='warp',
                no_components=COMPONENTS,
                learning_rate=LEARNING_RATE,
                item_alpha=ITEM_ALPHA,
                user_alpha=USER_ALPHA,
                random_state=42)

model.fit(interactions=sm_train_interactions,
          user_features=sm_user_features,
          item_features=sm_item_features,
          epochs=EPOCHS)

In [332]:
# оценим точность
train_precision = precision_at_k(model,
                                 test_interactions=sm_train_interactions,
                                 user_features=sm_user_features,
                                 item_features=sm_item_features,
                                 k=10).mean()



test_precision = precision_at_k(model,
                                test_interactions=sm_test_interactions,
                                user_features=sm_user_features,
                                item_features=sm_item_features,
                                k=10
                                ).mean()

train_precision, test_precision

(0.40562633, 0.105944805)

##TensorFlow Recommenders

In [1]:
!pip install tensorflow-recommenders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

In [45]:
# загрузим MovieLens
ratings_ds, ratings_info = tfds.load('movielens/100k-ratings',
                                     with_info=True,
                                     split='train')
movies_ds, movies_info = tfds.load('movielens/1m-movies',
                                    with_info=True,
                                    split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3883 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/1m-movies/incomplete.V59BXE_0.1.1/movielens-train.tfrecord*...: …

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/1m-movies/0.1.1. Subsequent calls will reuse this data.


In [46]:
# посмотрим информацию о датасете
movies_info

tfds.core.DatasetInfo(
    name='movielens',
    full_name='movielens/1m-movies/0.1.1',
    description="""
    This dataset contains a set of movie ratings from the MovieLens website, a movie
    recommendation service. This dataset was collected and maintained by [GroupLens]
    (https://grouplens.org/), a research group at the University of Minnesota. There
    are 5 versions included: "25m", "latest-small", "100k", "1m", "20m". In all
    datasets, the movies data and ratings data are joined on "movieId". The 25m
    dataset, latest-small dataset, and 20m dataset contain only movie data and
    rating data. The 1m dataset and 100k dataset contain demographic data in
    addition to movie and rating data.
    
    - "25m": This is the latest stable version of the MovieLens dataset. It is
    recommended for research purposes.
    - "latest-small": This is a small subset of the latest version of the MovieLens
    dataset. It is changed and updated over time by GroupLens.
    - "100k"

In [52]:
# посмотреть можно так
ds_r = tfds.as_dataframe(ratings_ds.take(5), ratings_info)
ds_r

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,7 (Drama),b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4 (doctor/health care),b'doctor',4.0,b'53211'
1,25.0,4 (Comedy)14 (Romance),b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5 (entertainment),b'entertainment',2.0,b'80525'
2,18.0,4 (Comedy),b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17 (student),b'student',4.0,b'55439'
3,50.0,5 (Crime)7 (Drama),b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4 (doctor/health care),b'healthcare',4.0,b'06472'
4,50.0,10 (Horror)16 (Thriller),b'895',b'Scream 2 (1997)',55.0,891409199,True,b'197',18 (technician/engineer),b'technician',3.0,b'75094'


In [53]:
ds_m = tfds.as_dataframe(movies_ds.take(5), movies_info)
ds_m

,movie_genres,movie_id,movie_title
0,5 (Crime)7 (Drama),b'1729',b'Jackie Brown (1997)'
1,7 (Drama),b'1486',"b'Quiet Room, The (1996)'"
2,4 (Comedy),b'3086',b'March of the Wooden Soldiers (a.k.a. Laurel & Hardy in Toyland) (1934)'
3,0 (Action),b'2965',"b'Omega Code, The (1999)'"
4,10 (Horror),b'2853',"b'Communion (a.k.a. Alice, Sweet Alice/Holy Terror) (1977)'"


In [5]:
# предварительная обработка данных
ratings = ratings.map(lambda x: {
    'movie_title': x['movie_title'],
    'user_id': x['user_id'],
})

movies = movies.map(lambda x: x['movie_title'])

In [6]:
# посмотрим, что это
type(ratings), type(movies)

(tensorflow.python.data.ops.map_op._MapDataset,
 tensorflow.python.data.ops.map_op._MapDataset)

In [14]:
z = ratings.splits['train'].num_examples
print(f'Train examples: {z}')

AttributeError: '_MapDataset' object has no attribute 'splits'

In [11]:
_ = tfds.as_dataframe(ratings.take(4), info)

NameError: name 'info' is not defined

In [10]:
ds, info = tfds.load('mnist', split='train', with_info=True)

tfds.as_dataframe(ds.take(4), info)

AttributeError: '_MapDataset' object has no attribute 'as_dataframe'

In [354]:
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
ds = ratings
#read the dataset into a tensorflow styled_dataframe
df = tfds.as_dataframe(ds)
#save the dataframe into csv file
df.to_csv('/content/covert_toxicity.csv')

#read the csv file as normal, then you have the df you need
import pandas as pd
file_path = '/content/covert_toxicity.csv'
df = pd.read_csv(file_path, header = 0, sep=",")
df

,Unnamed: 0,movie_title,user_id
0,0,"b""One Flew Over the Cuckoo's Nest (1975)""",b'138'
1,1,b'Strictly Ballroom (1992)',b'92'
2,2,"b'Very Brady Sequel, A (1996)'",b'301'
3,3,b'Pulp Fiction (1994)',b'60'
4,4,b'Scream 2 (1997)',b'197'
...,...,...,...
99995,99995,b'Star Trek: The Wrath of Khan (1982)',b'774'
99996,99996,"b'Game, The (1997)'",b'313'
99997,99997,"b""Wes Craven's New Nightmare (1994)""",b'262'
99998,99998,b'Alien (1979)',b'911'


In [372]:
import codecs


In [375]:
import codecs
byte_string = b"Do you want a slice of \xf0\x9f\x8d\x95?"
name_byte = codecs.decode(byte_string)
byte_string, name_byte

(b'Do you want a slice of \xf0\x9f\x8d\x95?', 'Do you want a slice of 🍕?')

In [373]:
df['user_id'] = df['user_id'].apply(lambda x: codecs.decode(x))
df

TypeError: decoding with 'utf-8' codec failed (TypeError: a bytes-like object is required, not 'str')

In [366]:
z = b"One Flew Over the Cuckoo's Nest (1975)"
x = z.decode("utf-8")
x

"One Flew Over the Cuckoo's Nest (1975)"